📘 NOTEBOOK 02 — PREPROCESAMIENTO DE DATOS

In [2]:
# ============================================
# 02 - PREPROCESAMIENTO DE DATOS
# Proyecto: MOCACI - Modelo de Clasificación de Amenazas Cibernéticas
# ============================================

# --------- MONTAR GOOGLE DRIVE ----------
from google.colab import drive
drive.mount('/content/drive')

# --------- IMPORTAR LIBRERÍAS ----------
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# --------- CARGAR DATASET RAW ----------
file_path = '/content/drive/MyDrive/Colab_Notebooks/Modulo 4/mocaci/data/raw/amenazas_ciberseguridad_v1.csv'

df = pd.read_csv(file_path)
print("Dataset cargado correctamente.\n")
df.head()

Mounted at /content/drive
Dataset cargado correctamente.



,timestamp,source_ip,country,destination_ip,protocol,port,packet_size,duration_sec,vpn_connection,firewall_action,cloudflare_flag,endpoint_detected,user_id,device_id,service_target,threat_type
0,2021-02-21 00:00:00,190.24.169.12,Colombia,10.0.52.71,ICMP,53,1017,2.68,0,allow,0,0,640,655,Portal de constancias,SQL Injection
1,2020-03-16 00:00:00,168.243.232.72,El Salvador,10.0.167.212,HTTP,53,259,0.14,0,allow,0,0,430,488,Portal de constancias,DDoS
2,2019-06-29 00:00:00,168.243.49.62,El Salvador,10.0.35.102,UDP,443,58,2.93,0,allow,0,0,214,686,Web institucional,SQL Injection
3,2022-04-18 00:00:00,83.32.92.189,España,10.0.49.121,UDP,80,1184,0.97,0,allow,0,0,92,228,Web institucional,Phishing
4,2024-09-24 00:00:00,34.178.20.136,Estados Unidos,10.0.28.178,UDP,110,1360,0.23,0,allow,0,1,508,324,Portal de constancias,Brute Force


🧹 Limpieza Inicial

In [3]:
# Mostrar valores nulos
print("Valores nulos por columna:\n")
print(df.isnull().sum())

# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"\nRegistros duplicados: {duplicados}")

# Eliminar duplicados si los hay
if duplicados > 0:
    df = df.drop_duplicates()
    print("Duplicados eliminados.")

Valores nulos por columna:

timestamp            0
source_ip            0
country              0
destination_ip       0
protocol             0
port                 0
packet_size          0
duration_sec         0
vpn_connection       0
firewall_action      0
cloudflare_flag      0
endpoint_detected    0
user_id              0
device_id            0
service_target       0
threat_type          0
dtype: int64

Registros duplicados: 0


🕒 Convertir columna de timestamp

In [4]:
# Convertir timestamp a formato datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], errors='coerce')

# Extraer características temporales útiles
df["year"] = df["timestamp"].dt.year
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day
df["hour"] = df["timestamp"].dt.hour

🧪 3. Selección de características relevantes (Feature Selection)

Eliminamos columnas que no aportan al modelo:

- timestamp: ya extraímos la información
- source_ip y destination_ip: demasiado específicas, no generalizan
- device_id: identificador único → no es útil

In [5]:
df = df.drop(columns=["timestamp", "source_ip", "destination_ip", "device_id"])
df.head()

,country,protocol,port,packet_size,duration_sec,vpn_connection,firewall_action,cloudflare_flag,endpoint_detected,user_id,service_target,threat_type,year,month,day,hour
0,Colombia,ICMP,53,1017,2.68,0,allow,0,0,640,Portal de constancias,SQL Injection,2021,2,21,0
1,El Salvador,HTTP,53,259,0.14,0,allow,0,0,430,Portal de constancias,DDoS,2020,3,16,0
2,El Salvador,UDP,443,58,2.93,0,allow,0,0,214,Web institucional,SQL Injection,2019,6,29,0
3,España,UDP,80,1184,0.97,0,allow,0,0,92,Web institucional,Phishing,2022,4,18,0
4,Estados Unidos,UDP,110,1360,0.23,0,allow,0,1,508,Portal de constancias,Brute Force,2024,9,24,0


🔤 4. Codificación de variables categóricas (Label Encoding)

Columnas categóricas:
- country
- protocol
- threat_type (⚠ objetivo del modelo → será y)

In [6]:
label_columns = ["country", "protocol", "threat_type"]

le = LabelEncoder()
for col in label_columns:
    df[col] = le.fit_transform(df[col])

print("Variables categóricas codificadas.")
df.head()

Variables categóricas codificadas.


,country,protocol,port,packet_size,duration_sec,vpn_connection,firewall_action,cloudflare_flag,endpoint_detected,user_id,service_target,threat_type,year,month,day,hour
0,1,2,53,1017,2.68,0,allow,0,0,640,Portal de constancias,7,2021,2,21,0
1,2,0,53,259,0.14,0,allow,0,0,430,Portal de constancias,1,2020,3,16,0
2,2,4,443,58,2.93,0,allow,0,0,214,Web institucional,7,2019,6,29,0
3,3,4,80,1184,0.97,0,allow,0,0,92,Web institucional,5,2022,4,18,0
4,4,4,110,1360,0.23,0,allow,0,1,508,Portal de constancias,0,2024,9,24,0


📊 5. Normalización de variables numéricas

Normalizamos:
- packet_size

In [7]:
scaler = StandardScaler()

df["packet_size"] = scaler.fit_transform(df[["packet_size"]])

🎯 6. División en Features (X) y Target (y)

In [8]:
X = df.drop(columns=["threat_type"])
y = df["threat_type"]

X.shape, y.shape

((3000, 15), (3000,))

🔀 7. Separación Train/Test (80/20)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("Train/Test Split realizado correctamente.")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

Train/Test Split realizado correctamente.
X_train: (2400, 15)
X_test: (600, 15)
y_train: (2400,)
y_test: (600,)


💾 8. Guardar dataset procesado para el modelo

In [10]:
processed_path = '/content/drive/MyDrive/Colab_Notebooks/Modulo 4/mocaci/data/processed/amenazas_ciberseguridad_prepared.csv'

processed_df = pd.concat([X, y], axis=1)
processed_df.to_csv(processed_path, index=False)

print(f"Dataset procesado guardado en:\n{processed_path}")

Dataset procesado guardado en:
/content/drive/MyDrive/Colab_Notebooks/Modulo 4/mocaci/data/processed/amenazas_ciberseguridad_prepared.csv
